# 2. Preparación de los Datos

Se implementó un pipeline de procesamiento de datos de acuerdo con los requisitos del modelo BERT.

In [71]:

import pandas as pd
from typing import List, Tuple

def cargar_datos() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Carga los datos desde archivos CSV.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]: 
        Tuple con cinco DataFrames: ventas, ventas_detalle, sucursales, usuarios, productos (momento de prueba, hay mas tablas que agregar después).
    """
    ventas = pd.read_csv('databd/farmacia_ventas.csv')
    ventas_detalle = pd.read_csv('databd/farmacia_venta_detalle.csv')
    sucursales = pd.read_csv('databd/general_sucursales.csv')
    usuarios = pd.read_csv('databd/general_usuarios.csv')
    productos = pd.read_csv('databd/producto_productos.csv')
    
    return ventas, ventas_detalle, sucursales, usuarios, productos

def procesar_datos(ventas: pd.DataFrame, ventas_detalle: pd.DataFrame, 
                   sucursales: pd.DataFrame, usuarios: pd.DataFrame, 
                   productos: pd.DataFrame) -> pd.DataFrame:
    """
    Procesa y une múltiples DataFrames en un único DataFrame consolidado.

    Args:
        ventas (pd.DataFrame): DataFrame con los datos de ventas.
        ventas_detalle (pd.DataFrame): DataFrame con los detalles de las ventas.
        sucursales (pd.DataFrame): DataFrame con la información de las sucursales.
        usuarios (pd.DataFrame): DataFrame con la información de los usuarios.
        productos (pd.DataFrame): DataFrame con la información de los productos.

    Returns:
        pd.DataFrame: DataFrame consolidado.
    """
    ventas['venta_id'] = ventas['venta_id'].astype(str)
    ventas_detalle['venta_id'] = ventas_detalle['venta_id'].astype(str)
    ventas['usuario_id'] = ventas['usuario_id'].astype(str)
    usuarios['user_id'] = usuarios['user_id'].astype(str)
    ventas['sucursal_id'] = ventas['sucursal_id'].astype(str)
    sucursales['suc_id'] = sucursales['suc_id'].astype(str)
    ventas_detalle['prod_id'] = ventas_detalle['prod_id'].astype(str)
    productos['prod_id'] = productos['prod_id'].astype(str)

    df = ventas.merge(ventas_detalle, on='venta_id', how='inner')
    df = df.merge(sucursales, left_on='sucursal_id', right_on='suc_id', how='inner')
    df = df.merge(usuarios, left_on='usuario_id', right_on='user_id', how='inner')
    df = df.merge(productos, on='prod_id', how='inner')
   
    
    return df

def preparar_textos(df: pd.DataFrame) -> List[str]:
    """
    Prepara una lista de textos a partir de un DataFrame consolidado.

    Args:
        df (pd.DataFrame): DataFrame consolidado.

    Returns:
        List[str]: Lista de textos preparados para el modelo.
    """
    textos = df.apply(lambda row: f"Producto: {row['prod_nombre']}, Sucursal: {row['suc_nombre']}, Vendedor: {row['user_name']}", axis=1)
    return textos.tolist()

Cargamos y procesamos los datos

In [72]:
ventas, ventas_detalle, sucursales, usuarios, productos = cargar_datos() # Cargar datos desde archivos CSV 

In [73]:
df_procesado = procesar_datos(ventas, ventas_detalle, sucursales, usuarios, productos) # Procesar datos y unirlos en un único DataFrame

In [74]:
textos = preparar_textos(df_procesado) # Preparar textos para el modelo de lenguaje natural (NLP)

In [75]:
textos   # Mostrar textos preparados  para el modelo de lenguaje natural (NLP)

['Producto: KIT CASA MELASMA VERDE, Sucursal: Central, Vendedor: master',
 'Producto: KIT CASA MELASMA VERDE, Sucursal: Central, Vendedor: master',
 'Producto: FORMULA 3: TRI L, Sucursal: Central, Vendedor: master',
 'Producto: KIT CASA MELASMA VERDE, Sucursal: Central, Vendedor: master',
 'Producto: HYALU B5 SERUM 30ML, Sucursal: Central, Vendedor: master',
 'Producto: HYALU B5 SERUM 30ML, Sucursal: Central, Vendedor: master',
 'Producto: PHOTODERM SPOT-AGE  SPF  50+ 40ML\t, Sucursal: Central, Vendedor: master',
 'Producto: CETAPHIL PRO AD CONTROL , Sucursal: Central, Vendedor: master',
 'Producto: EFFACLAR BB BLUR  30ML\t, Sucursal: Central, Vendedor: master',
 'Producto: PERSPIREX COMFORT ROLL ON X 20 ML\t, Sucursal: Central, Vendedor: master',
 'Producto: PERSPIREX COMFORT ROLL ON X 20 ML\t, Sucursal: Central, Vendedor: master',
 'Producto: PHOTODERM AQUAFLUIDE DORE TE40ML, Sucursal: Central, Vendedor: master',
 'Producto: PHOTODERM COVER TOUCH FL DOREE 50 + T 40GR\t, Sucursal: Cen

# 3. Integración del Modelo Encoder

## 3.1. Instalación de Dependencias

In [76]:
%pip install transformers  # Instalar la librería transformers de Hugging Face para trabajar con modelos de lenguaje natural (NLP)


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 3.2. Carga del Modelo Encoder Preentrenado

### Generación de Embeddings

In [77]:
from transformers import BertTokenizer, BertModel
import torch

class BertEmbedder:
    def __init__(self, model_name: str = 'bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
    
    def generar_embeddings(self, textos: List[str]) -> torch.Tensor:
        inputs = self.tokenizer(textos, return_tensors='pt', padding=True, truncation=True, max_length=128)
        outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state
        return embeddings

embedder = BertEmbedder()
embeddings = embedder.generar_embeddings(textos)
print(embeddings.shape)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

torch.Size([62, 54, 768])


In [78]:
#REVISAMOS EL DATASET PROCESADO ANTERIORMENTE
df_procesado

,venta_id,venta_fecha,cliente_id,venta_proceso,usuario_id,sucursal_id_x,venta_estado,det_id,prod_id,detalle_medida,...,prod_nombre,prod_descripcion,precio_venta_y,med_id,precio_unitario,medida_unitario,cantidad_paquete,imagen_nombre,cat_id,prod_estado
0,147,2024-07-12,14,PROFORMA,1,1,1,187,18,1,...,KIT CASA MELASMA VERDE,FOTODINAMICA,420.0,1,420.0,1,420.0,NaN,1,1
1,148,2024-07-12,14,PROFORMA,1,1,1,188,18,1,...,KIT CASA MELASMA VERDE,FOTODINAMICA,420.0,1,420.0,1,420.0,NaN,1,1
2,149,2024-07-12,14,PROFORMA,1,1,1,189,23,1,...,FORMULA 3: TRI L,GALDERMA\t,209.0,1,209.0,1,1.0,NaN,1,1
3,150,2024-07-12,14,PROFORMA,1,1,1,190,18,1,...,KIT CASA MELASMA VERDE,FOTODINAMICA,420.0,1,420.0,1,420.0,NaN,1,1
4,151,2024-07-12,14,PROFORMA,1,1,1,191,24,9,...,HYALU B5 SERUM 30ML,LA ROCHE P\tFRASCO\t S/207.00,207.0,9,207.0,9,1.0,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,190,2024-08-01,14,PROFORMA,1,1,1,244,27,7,...,"FISSERUM HYALURONIC 100, 30 ML\t",FISSIONLAB,176.0,7,1.0,7,1.0,NaN,1,1
58,191,2024-08-02,22,PAGADO,1,1,1,245,30,1,...,"FISSERUM NIGTH RETINOL 3 COMPLEX, SQUALANE, HY...",FISSIONLAB\t,170.0,1,170.0,1,1.0,NaN,1,1
59,191,2024-08-02,22,PAGADO,1,1,1,246,34,1,...,PHOTODERM AQUAFLUIDE DORE TE40ML,BIODERMA,166.0,1,166.0,1,1.0,NaN,1,1
60,192,2024-08-02,24,PAGADO,1,1,1,247,50,1,...,PHOTODERM AKN MAT SPF30,40ML\tBIODERMA\t,110.0,1,110.0,1,1.0,NaN,1,1


## Preparación para Fine-Tuning y Entrenamiento del Modelo

Añadir etiquetas al DataFrame procesado

In [86]:
df_procesado['label'] = df_procesado['venta_proceso'].apply(lambda x: 1 if x.strip().lower() == 'pagado' else 0)  # Crear una columna 'label' con valores 1 si la venta fue completada y 0 si no

In [80]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch

# Definir la clase Dataset
class VentasDataset(Dataset):
    """
    Clase para manejar los datos de ventas en un conjunto de datos.

    Args:
        encodings (dict): Diccionario con las codificaciones de los textos.
        labels (List[int]): Lista de etiquetas correspondientes a los textos.
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Args:
            idx (int): Índice del elemento a obtener.

        Returns:
            dict: Diccionario con las codificaciones y la etiqueta correspondiente.
        """
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Obtiene el tamaño del conjunto de datos.

        Returns:
            int: Número de elementos en el conjunto de datos.
        """
        return len(self.labels)

# Tokenización y preparación de datos
train_texts, val_texts, train_labels, val_labels = train_test_split(textos, df_procesado['label'].tolist(), test_size=0.2)  #Dividimos los datos en conjuntos de entrenamiento y evaluación
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #Tokenizamos los textos con el tokenizador de BERT
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128) #Codificamos los textos de entrenamiento con el tokenizador de BERT
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128) #Codificamos los textos de evaluación con el tokenizador de BERT

# Crear conjuntos de datos
train_dataset = VentasDataset(train_encodings, train_labels)    #Conjunto de datos de entrenamiento
val_dataset = VentasDataset(val_encodings, val_labels)  #Conjunto de datos de evaluación

# Configurar el modelo de clasificación
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')  #Modelo de clasificación de texto basado en BERT con 2 clases de salida

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',           # Carpeta de salida
    num_train_epochs=10,               # Número de épocas de entrenamiento
    per_device_train_batch_size=8,    # Tamaño del lote de entrenamiento
    per_device_eval_batch_size=8,     # Tamaño del lote de evaluación
    warmup_steps=500,                 # Pasos de calentamiento
    weight_decay=0.01,                # Decaimiento de peso
    logging_dir='./logs',             # Carpeta de registros
    logging_steps=10,
    save_strategy="no",               # Deshabilitar guardado automático
)

# Crear el entrenador
trainer = Trainer(  #Entrenamos el modelo de clasificación
    model=model,    #Modelo de clasificación a entrenar
    args=training_args, #Argumentos de entrenamiento
    train_dataset=train_dataset,    #Conjunto de datos de entrenamiento
    eval_dataset=val_dataset    #Conjunto de datos de evaluación
)

# Entrenar el modelo
trainer.train()

# Evaluar el rendimiento del modelo
results = trainer.evaluate()
print(results)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

  0%|          | 0/70 [00:00<?, ?it/s]

{'loss': 0.8727, 'grad_norm': 13.39725399017334, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.43}
{'loss': 0.7408, 'grad_norm': 11.317423820495605, 'learning_rate': 2.0000000000000003e-06, 'epoch': 2.86}
{'loss': 0.45, 'grad_norm': 8.45865535736084, 'learning_rate': 3e-06, 'epoch': 4.29}
{'loss': 0.3748, 'grad_norm': 8.265202522277832, 'learning_rate': 4.000000000000001e-06, 'epoch': 5.71}
{'loss': 0.2515, 'grad_norm': 5.215757369995117, 'learning_rate': 5e-06, 'epoch': 7.14}
{'loss': 0.1714, 'grad_norm': 2.898374557495117, 'learning_rate': 6e-06, 'epoch': 8.57}
{'loss': 0.0858, 'grad_norm': 2.273519277572632, 'learning_rate': 7.000000000000001e-06, 'epoch': 10.0}
{'train_runtime': 10.5693, 'train_samples_per_second': 46.361, 'train_steps_per_second': 6.623, 'train_loss': 0.42102279237338475, 'epoch': 10.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.04464004933834076, 'eval_runtime': 0.1744, 'eval_samples_per_second': 74.525, 'eval_steps_per_second': 11.465, 'epoch': 10.0}


RESULTADO: VEMOS QUE SE REALIZÓ CON ÉXITO Y OBTENEMOS ESTO:\
   •	Pérdida de Entrenamiento (train_loss): 0.42102279237338475\
	•	Pérdida de Evaluación (eval_loss): 0.04464004933834076


Los resultados muestran que el modelo ha aprendido de los datos de entrenamiento y generaliza bien a los datos de validación, como se evidencia por la baja pérdida de evaluación. Esto sugiere que los embeddings generados son representaciones útiles de mis datos y pueden ser utilizados después correctamente.


Guardamos el modelo y definimos el max_shard_size apra que no haya errores.

In [81]:
from transformers import BertForSequenceClassification, BertTokenizer

# Definir max_shard_size como un entero
max_shard_size = 10 * 1024 * 1024 * 1024  # 10GB en bytes

# Guardar el modelo
model.save_pretrained('./results', max_shard_size=max_shard_size)
tokenizer.save_pretrained('./results')

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json')

Verificamos si el directorio ./results existe y tiene permisos de escritura

In [82]:
import os

if not os.path.exists('./results'):
    os.makedirs('./results')

In [83]:
#CARGAMOS EL MODELO ENTRENADO
modelentrenado = BertForSequenceClassification.from_pretrained('./results')
tokenizerentrenado = BertTokenizer.from_pretrained('./results')

# Ejemplo solo de prueba
inputs = tokenizerentrenado("Texto de ejemplo ", return_tensors="pt")
outputs = modelentrenado(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
print(predictions)

tensor([0])
